TO-DO:
- add merge_geom parameter to dissolve to automatically pull that using the created relation
- add cut and interpolate aggregators

In [1]:
import linref as lr
import linref_011 as lr011
from linref.events import EventsData
from linref.events import common, selection, modify, utility, analyze, geometry
import numpy as np
import pandas as pd
from shapely.geometry import LineString
import scipy.sparse as sparse
import geopandas as gpd
import linref as lr
import warnings

In [2]:
lr.set_default_lrs(key_col=['county', 'route'], loc_col='loc', beg_col='beg', end_col='end', geom_col='geometry', closed='left_mod')

df = gpd.read_file('testing/data/linear-events-3.geojson').set_index('index')
df.lr

LRS_Accessor with linear referencing system (LRS) objects:
- LRS(key_col=['county', 'route'], loc_col=loc, beg_col=beg, end_col=end, geom_col=geometry, geom_m_col=None, closed=left_mod)

In [3]:
df.lr.add_chaining(inplace=True)
df.lr

LRS_Accessor with linear referencing system (LRS) objects:
- LRS(key_col=['county', 'route', 'chain'], loc_col=loc, beg_col=beg, end_col=end, geom_col=geometry, geom_m_col=None, closed=left_mod)

In [4]:
df.lr.add_geom_m(inplace=True)
df.lr

LRS_Accessor with linear referencing system (LRS) objects:
- LRS(key_col=['county', 'route', 'chain'], loc_col=loc, beg_col=beg, end_col=end, geom_col=geometry, geom_m_col=None, closed=left_mod)
- LRS(key_col=['county', 'route', 'chain'], loc_col=loc, beg_col=beg, end_col=end, geom_col=geometry, geom_m_col=geometry_m, closed=left_mod)

In [5]:
dissolved, relation = df.lr.dissolve(return_relation=True)
dissolved

,county,route,chain,beg,end,dissolved_index,geometry,geometry_m
index,,,,,,,,
0,A,Main St,0,0.0,0.4,[7],"LINESTRING (0 0, 0.8 0)","LINESTRING M (0.0 0.0 0.0, 0.8 0.0 0.4) # linr..."
1,A,Main St,1,0.5,3.0,"[6, 9]","LINESTRING (1 0, 1 1, 2 1)","LINESTRING M (1.0 0.0 0.5, 1.0 1.0 1.1, 2.0 1...."
2,A,1st Ave,0,10.0,13.0,"[4, 3]","LINESTRING (10 0, 12 0, 13 0)","LINESTRING M (10.0 0.0 10.0, 12.0 0.0 12.0, 13..."
3,B,2nd Ave,0,13.0,18.1,"[2, 0, 1, 10, 13]","LINESTRING (13 0, 13 1, 14 1, 17 1, 17 2, 17 3)","LINESTRING M (13.0 0.0 13.0, 13.0 1.0 13.1, 14..."
4,C,Sycamore School Rd,0,1.8,3.0,[8],"LINESTRING (1 3, 1 4)","LINESTRING M (1.0 3.0 1.8, 1.0 4.0 3.0) # linr..."


In [6]:
relation[['volume', 'class']].mode()

array([[100, 'B'],
       [200, 'A'],
       [400, 'A'],
       [300, 'B'],
       [800, 'A']], dtype=object)

# Resegmentation

Return relation from resegmentation

In [10]:
df.lr.resegment(length=.25, fill='balance')

,county,route,chain,beg,end,volume,class
7,A,Main St,0,0.00,0.25,100,B
7,A,Main St,0,0.25,0.40,100,B
6,A,Main St,1,0.50,0.75,200,A
6,A,Main St,1,0.75,1.10,200,A
9,A,Main St,1,1.10,1.35,200,A
9,A,Main St,1,1.35,1.60,200,A
9,A,Main St,1,1.60,1.85,200,A
9,A,Main St,1,1.85,2.10,200,A
9,A,Main St,1,2.10,2.35,200,A
9,A,Main St,1,2.35,2.60,200,A


In [8]:
events = base.EventsData(index=['a', 'b', 'c', 'd', 'e'], begs=[0.0, 0.1, 1.2, 1.7, 2.6], ends=[0.1, 1.2, 1.7, 2.6, 5.0])

NameError: name 'base' is not defined

In [ ]:
events.to_frame()

,beg,end
a,0.0,0.1
b,0.1,1.2
c,1.2,1.7
d,1.7,2.6
e,2.6,5.0


In [ ]:
events = base.EventsData(begs=[0.0, 0.1, 1.2, 1.7, 2.6], ends=[0.1, 1.2, 1.7, 2.6, 5.0])
for fill in common.segment_fill_all:
    new_events = resegment(events, length=1, fill=fill)

In [ ]:
display(events)
for fill in common.segment_fill_all:
    new_events = resegment(events, length=1, fill=fill)
    display(fill)
    display(new_events)
new_events

0, (0.000, 0.100]
1, (0.100, 1.200]
2, (1.200, 1.700]
3, (1.700, 2.600]
4, (2.600, 5.000]
EventsData(5 ungrouped, monotonic linear events, closed=right)

'right'

0, group(0) (0.000, 1.000]
1, group(0) (0.100, 1.100]
1, group(0) (1.100, 2.100]
2, group(0) (1.200, 2.200]
3, group(0) (1.700, 2.700]
4, group(0) (2.600, 3.600]
4, group(0) (3.600, 4.600]
4, group(0) (4.600, 5.600]
EventsData(8 grouped, monotonic linear events, closed=right)

'cut'

0, group(0) (0.000, 0.100]
1, group(0) (0.100, 1.100]
1, group(0) (1.100, 1.200]
2, group(0) (1.200, 1.700]
3, group(0) (1.700, 2.600]
4, group(0) (2.600, 3.600]
4, group(0) (3.600, 4.600]
4, group(0) (4.600, 5.000]
EventsData(8 grouped, monotonic linear events, closed=right)

...

'balance'

0, group(0) (0.000, 0.100]
1, group(0) (0.100, 1.200]
2, group(0) (1.200, 1.700]
3, group(0) (1.700, 2.600]
4, group(0) (2.600, 3.600]
4, group(0) (3.600, 5.000]
EventsData(6 grouped, monotonic linear events, closed=right)

'extend'

0, group(0) (0.000, 0.100]
1, group(0) (0.100, 1.200]
2, group(0) (1.200, 1.700]
3, group(0) (1.700, 2.600]
4, group(0) (2.600, 3.600]
4, group(0) (3.600, 5.000]
EventsData(6 grouped, monotonic linear events, closed=right)

0, group(0) (0.000, 0.100]
1, group(0) (0.100, 1.200]
2, group(0) (1.200, 1.700]
3, group(0) (1.700, 2.600]
4, group(0) (2.600, 3.600]
4, group(0) (3.600, 5.000]
EventsData(6 grouped, monotonic linear events, closed=right)